In [1]:
#!pip install scikit-network
#!pip install dgl

In [19]:
import ChebNet
import CORA
import utils

import dgl
import torch

In [12]:
# Reload when files are changed
import importlib
importlib.reload(ChebNet)
importlib.reload(CORA)
importlib.reload(utils)

<module 'utils' from '/Users/alice/Documents/GIT/CayleyNets/utils.py'>

In [18]:
# Checking torch and device

# Torch version
print(torch.__version__)

# Is MPS even available? macOS 12.3+
print(torch.backends.mps.is_available())

# Was the current version of PyTorch built with MPS activated?
print(torch.backends.mps.is_built())

2.1.1
True
True


In [14]:
cora = CORA.CORA()

Parsing files...
Done.


In [15]:
# device = torch.device('mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [16]:
train_mask, test_mask, val_mask = utils.split_train_test_val(cora.n_nodes)

In [17]:
# Hyperparameters
in_feats = cora.n_features
n_classes = cora.n_classes
n_hidden = 16
n_layers = 1 # number of hidden+output layers
k = 5 # Chebyshev polynomial order

In [20]:
# Graph as an object
graph = dgl.from_scipy(cora.adjacency)

# Features and labels as tensors
features = torch.Tensor(cora.features).to(device)
labels = torch.Tensor(cora.labels).long().to(device)

# Masks as tensors
train_mask = torch.Tensor(train_mask).bool().to(device)
test_mask = torch.Tensor(test_mask).bool().to(device)
val_mask = torch.Tensor(val_mask).bool().to(device)

In [22]:
# Define model
model = ChebNet.ChebNet(graph, in_feats, n_classes, n_hidden, n_layers, k, bias=True).to(device)
model

ChebNet(
  (layers): ModuleList(
    (0): ChebConv(
      (linear): Linear(in_features=7165, out_features=16, bias=True)
    )
    (1): ChebConv(
      (linear): Linear(in_features=80, out_features=7, bias=True)
    )
  )
)

In [23]:
print(f'Number of parameters: {sum(p.numel() for p in model.parameters())}')
print(f'Number of parameters: {sum(p.numel() for p in model.layers[0].parameters())}')
print(f'Number of parameters: {sum(p.numel() for p in model.layers[1].parameters())}')

Number of parameters: 115223
Number of parameters: 114656
Number of parameters: 567


In [24]:
# Optimizer
lr = 5e-3
weight_decay = 5e-4
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

In [25]:
# Train the model
batch_size = 32
epochs = 100
# save loss values for plotting
loss_values = []
for e in range(epochs):
    # Compute output
    output = model(features)

    # Compute loss
    logp = F.log_softmax(output, 1)
    loss = F.nll_loss(logp[train_mask], labels[train_mask])
    loss_values.append(loss.item())

    # Perform backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print loss
    print(f'Epoch: {e:03d} | Loss: {loss:.4f}')

Epoch: 000 | Loss: 1.9309
Epoch: 001 | Loss: 1.8730
Epoch: 002 | Loss: 1.8086
Epoch: 003 | Loss: 1.7481
Epoch: 004 | Loss: 1.6782
Epoch: 005 | Loss: 1.6020
Epoch: 006 | Loss: 1.5267
Epoch: 007 | Loss: 1.4562
Epoch: 008 | Loss: 1.3918
Epoch: 009 | Loss: 1.3350
Epoch: 010 | Loss: 1.2884
Epoch: 011 | Loss: 1.2517
Epoch: 012 | Loss: 1.2201
Epoch: 013 | Loss: 1.1930
Epoch: 014 | Loss: 1.1701
Epoch: 015 | Loss: 1.1497
Epoch: 016 | Loss: 1.1309
Epoch: 017 | Loss: 1.1133
Epoch: 018 | Loss: 1.0968
Epoch: 019 | Loss: 1.0814
Epoch: 020 | Loss: 1.0671
Epoch: 021 | Loss: 1.0534
Epoch: 022 | Loss: 1.0407
Epoch: 023 | Loss: 1.0293
Epoch: 024 | Loss: 1.0186
Epoch: 025 | Loss: 1.0087
Epoch: 026 | Loss: 0.9997
Epoch: 027 | Loss: 0.9913
Epoch: 028 | Loss: 0.9835
Epoch: 029 | Loss: 0.9763
Epoch: 030 | Loss: 0.9698
Epoch: 031 | Loss: 0.9639
Epoch: 032 | Loss: 0.9586
Epoch: 033 | Loss: 0.9539
Epoch: 034 | Loss: 0.9500
Epoch: 035 | Loss: 0.9465
Epoch: 036 | Loss: 0.9435
Epoch: 037 | Loss: 0.9408
Epoch: 038 |